In [10]:
import os, sys, re
import sqlite3, csv
import argparse
from tqdm.notebook import tqdm
import json
import pandas as pd
import requirements
# %pip install ipywidgets

In [2]:
args = {
    "python_files": "/home/gjungwirth/data/02_data/03_analyses/library_versions/requirements-files",
    "node_deps": "/home/gjungwirth/data/02_data/03_analyses/library_versions/packagejson-deps.csv",
    "db": "/home/gjungwirth/data/02_data/results.db"
}

In [3]:
df_node = pd.read_csv(args["node_deps"], names=['name','version','type','filepath'])

df_node[["repo","filepath_new"]] = df_node.filepath.str.split('/', n=2, expand=True).get([1,2])

df_node["filename_only"] = df_node.filepath_new.str.split("/").str[-1]

df_node

,name,version,type,filepath,repo,filepath_new,filename_only
0,bent,^7.3.10,dependencies,repos/monkpow_dotfiles/github/package.json,monkpow_dotfiles,github/package.json,package.json
1,request,^2.88.2,dependencies,repos/monkpow_dotfiles/github/package.json,monkpow_dotfiles,github/package.json,package.json
2,underscore-plus,^1.2.1,dependencies,repos/tom-lpsd_confs/atom/packages/emacs-mode/...,tom-lpsd_confs,atom/packages/emacs-mode/package.json,package.json
3,js-beautify,1.7.4,dependencies,repos/glenngillen_dotfiles/.vscode/extensions/...,glenngillen_dotfiles,.vscode/extensions/aeschli.vscode-css-formatte...,package.json
4,highlight.js,^9.13.1,dependencies,repos/glenngillen_dotfiles/.vscode/extensions/...,glenngillen_dotfiles,.vscode/extensions/bierner.markdown-preview-gi...,package.json
...,...,...,...,...,...,...,...
145618,eslint-config-dmitmel,dmitmel/eslint-config-dmitmel,devDependencies,repos_part2/keanuplayz_dotfiles/script-resourc...,keanuplayz_dotfiles,script-resources/markdown2htmldoc/package.json,package.json
145619,eslint-config-prettier,*,devDependencies,repos_part2/keanuplayz_dotfiles/script-resourc...,keanuplayz_dotfiles,script-resources/markdown2htmldoc/package.json,package.json
145620,eslint-plugin-node,*,devDependencies,repos_part2/keanuplayz_dotfiles/script-resourc...,keanuplayz_dotfiles,script-resources/markdown2htmldoc/package.json,package.json
145621,eslint-plugin-prettier,*,devDependencies,repos_part2/keanuplayz_dotfiles/script-resourc...,keanuplayz_dotfiles,script-resources/markdown2htmldoc/package.json,package.json


In [39]:
import os
os.getcwd()
os.chdir('/home/gjungwirth/data/02_data/')

req_files = []
with open(args['python_files']) as f:
    req_files = [line.rstrip() for line in f]
    # req_files[0:1]

error_cnt=0
all_reqs = []
for reqfile in req_files:
    try:
        with open(reqfile) as f:
            for req in requirements.parse(f):
                name = req.name
                spec_string = ",".join(map("".join, req.specs))
                (_, repo, filepath_new) = reqfile.split('/', maxsplit=2)
                filename_only = filepath_new.split('/')[-1]
                #print("{}, spec: {} repo: {}, filepath_new: {}, filename: {}".format(name, spec_string, repo, filepath_new, filename_only))
                all_reqs.append([
                    name, spec_string, reqfile, repo, filepath_new, filename_only
                ])
    except:
        error_cnt+=1

print("Errors: {}/{}".format(error_cnt,len(req_files)))

df_python = pd.DataFrame(all_reqs, columns=["name","version","filepath","repo","filepath_new","filename_only"])

/home/gjungwirth/data/02_data/.venv/lib/python3.9/site-packages/requirements/parser.py:44: UserWarning: Private repos not supported. Skipping.
  warnings.warn('Private repos not supported. Skipping.')
Errors: 36/1340


In [40]:
display(df_node.describe())
display(df_python.describe())   

,name,version,type,filepath,repo,filepath_new,filename_only
count,145522,145261,145623,145623,145623,145623,145623
unique,5597,4630,2,20578,585,12671,1
top,mocha,^1.0.0,devDependencies,repos_part2/alisonamerico_dotfiles/.vscode/ext...,rafaelconde_dotfiles,package.json,package.json
freq,6103,6456,96869,285,21410,940,145623


,name,version,filepath,repo,filepath_new,filename_only
count,16321,16410,16410,16410,16410,16410
unique,3027,1913,1286,1043,592,3
top,flake8,,repos_part2/nraw_dotfiles/requirements.txt,nraw_dotfiles,requirements.txt,requirements.txt
freq,255,4750,499,499,7673,16370


In [43]:
db = sqlite3.connect(args['db'])

db.execute('''
CREATE TABLE IF NOT EXISTS dependency (
    dependency_id INTEGER PRIMARY KEY,
    file_id INTEGER NOT NULL,
    type TEXT NOT NULL, -- (javascript, python)
    name TEXT NOT NULL,
    version TEXT NOT NULL,
    isDev BOOLEAN NOT NULL,
    -- tags TEXT NOT NULL,
    FOREIGN KEY (file_id) REFERENCES file (file_id),
    UNIQUE (file_id, type, name, version)
);
''')
db.commit()

In [42]:
# only a test to find historic files (hopefully they are zero here)

num_total = 0
num_empty = 0

for row in df_python.drop_duplicates(subset=["filepath_new","repo"]).itertuples():
    # print(leak['file'],repo_folder)
    cur = db.execute('''
        SELECT f.file_id
        FROM file f JOIN repo r ON f.repo_id=r.repo_id
        WHERE
        f.file_path = ? AND
        r.folder_name = ?
        ''', 
        (row.filepath_new, row.repo)
        )
    if not cur.fetchall():
        num_empty+=1
    num_total+=1

print("Empty: {}/{} = {}%".format(num_empty, num_total, int(num_empty*100/num_total)))

Empty: 143/1286 = 11%


In [46]:
df_node.shape[0]

145623

In [49]:
# Insert Node Dependencies (easier separate)

pbar = tqdm(total=df_node.shape[0])

for row in df_node.itertuples():
    
    db.execute('''
        INSERT OR IGNORE INTO file
        (repo_id, file_name, file_path, is_deleted, size, sha, mime, no_lines)
        VALUES
        (
            (SELECT repo_id FROM repo WHERE folder_name = ?),
            ?, ?,
            TRUE,
            -1, "", "", -1
        );
        ''',
        (row.repo, row.filename_only, row.filepath_new)
        )
    db.commit()
    db.execute('''
        INSERT OR IGNORE INTO dependency
        (file_id, type, name, version, isDev)
        VALUES
        (
            (SELECT file_id FROM file f JOIN repo r ON f.repo_id=r.repo_id WHERE r.folder_name = ? AND f.file_path = ?),
            "javascript",
            ?,
            ?,
            ?
        );''',
        (
            row.repo, row.filepath_new,
            row.name,
            row.version,
            (row.type == "devDependencies")
        ))
    db.commit()
    pbar.update()



In [50]:
# Insert Python Dependencies

pbar = tqdm(total=df_python.shape[0])

for row in df_python.itertuples():
    
    db.execute('''
        INSERT OR IGNORE INTO file
        (repo_id, file_name, file_path, is_deleted, size, sha, mime, no_lines)
        VALUES
        (
            (SELECT repo_id FROM repo WHERE folder_name = ?),
            ?, ?,
            TRUE,
            -1, "", "", -1
        );
        ''',
        (row.repo, row.filename_only, row.filepath_new)
        )
    db.commit()
    db.execute('''
        INSERT OR IGNORE INTO dependency
        (file_id, type, name, version, isDev)
        VALUES
        (
            (SELECT file_id FROM file f JOIN repo r ON f.repo_id=r.repo_id WHERE r.folder_name = ? AND f.file_path = ?),
            "python",
            ?,
            ?,
            FALSE
        );''',
        (
            row.repo, row.filepath_new,
            row.name,
            row.version
        ))
    db.commit()
    pbar.update()



In [51]:
db.close()